In [174]:
import wrds
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Geterate Stock and Factor Return

In [175]:
db = wrds.Connection()

Enter your WRDS username [jay]:rachelzq
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [235]:
stocksql = "select date, cusip, prc as price, cfacpr as adjfactor, vol \
            from crsp.dsf where date between '2015-01-01' and '2018-12-31' \
            and vol is not NULL \
            and cusip in (select cusip from crsp.dsf where date='2015-01-02' order by vol DESC limit 200)"

stock_dt = db.raw_sql(stocksql)
stock_dt['adjprice'] = stock_dt.price * stock_dt.adjfactor
stock_dt.set_index(['date', 'cusip'], inplace=True)
stock_dt.sort_index(inplace=True)
stock_dt['ret'] = stock_dt.price.groupby('cusip').pct_change()
stock_dt.dropna(inplace=True)

stock_dt.to_csv('stock.csv')

In [236]:
def read_factor(filename):
    dt = pd.read_csv(filename)
    dt = dt[(dt.date > 20150103) & (dt.date <= 20181231)]
    dt.date = pd.to_datetime(dt.date, format='%Y%m%d')
    dt.set_index('date', inplace=True)
    return dt

fama_dt = read_factor('fama.csv')
STREV_dt = read_factor('ShrtermReversal.csv')
LTREV_dt = read_factor('LongtermReversal.csv')
industry_dt = read_factor('industry.csv')

momentum_sql = "select date, umd as momentum \
       from ff.factors_daily where date between '2015-01-05' and '2018-12-31'"
mom = db.raw_sql(momentum_sql)

mom.set_index('date', inplace=True)
fama_dt['MOM'] = mom.momentum

factors = fama_dt.join([ShrttermReversal_dt, LongtermReversal_dt, industry_dt], how='inner')
factors

,Mkt-RF,SMB,HML,RMW,CMA,MOM,ST_Rev,LT_Rev,Agric,Food,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
date,,,,,,,,,,,,,,,,,,,,,
2015-01-05,-1.84,0.25,-0.65,0.16,-0.12,0.0119,-0.56,-0.38,-0.90,-1.01,...,-1.31,-2.47,-2.01,-0.99,-1.06,-1.24,-1.43,-0.16,-1.50,-0.88
2015-01-06,-1.04,-0.78,-0.26,0.54,0.04,0.0048,-0.32,-0.31,-1.03,-0.68,...,-1.25,-2.33,-1.69,-1.58,-1.00,-1.22,-1.33,-1.06,-1.65,-1.24
2015-01-07,1.19,0.16,-0.62,0.24,-0.14,0.0093,-0.57,-0.17,1.29,1.59,...,-0.35,0.20,0.34,3.30,3.42,0.40,0.68,0.36,1.18,0.87
2015-01-08,1.81,-0.10,-0.29,0.10,-0.20,0.0012,0.18,-0.16,1.05,1.35,...,2.26,1.88,1.03,1.96,1.30,0.87,1.50,1.72,0.87,1.38
2015-01-09,-0.85,-0.02,-0.49,-0.12,-0.18,0.0004,-0.35,0.07,1.09,-0.52,...,-0.59,-1.04,-1.03,-1.91,-0.46,-1.06,-1.28,0.06,-0.64,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-24,-2.55,0.90,-0.44,-0.36,-0.36,-0.0021,0.09,-0.45,-3.24,-2.85,...,-2.71,-2.08,-2.03,-0.75,-1.60,-1.60,-2.01,-0.85,-1.44,-1.69
2018-12-26,5.06,-0.18,-1.01,-0.19,-1.07,0.0001,1.06,0.44,0.26,2.73,...,3.00,3.57,3.24,5.00,2.23,3.11,3.55,2.43,3.39,1.78
2018-12-27,0.78,-0.70,-0.15,-0.13,0.10,0.0029,-0.55,-0.47,-1.33,0.62,...,1.54,-0.17,-0.27,-1.04,-0.23,-0.64,0.72,-0.38,0.94,-0.18


In [237]:
stock_dt

price  adjfactor         vol   adjprice       ret
date       cusip                                                          
2015-01-05 00154710   5.610000        1.0  10697057.0   5.610000 -0.060301
           00162Q86  17.180000        1.0   6900730.0  17.180000 -0.027180
           00206R10  33.549999        1.0  30429132.0  33.549999 -0.009448
           00448Q20  13.320000        1.0   4699604.0  13.320000 -0.000750
           00790310   2.660000        1.0   8878176.0   2.660000 -0.011152
...                        ...        ...         ...        ...       ...
2018-12-31 G6543110   2.620000        1.0   5020794.0   2.620000 -0.040293
           G7998G10   9.750000        1.0    316048.0   9.750000  0.028481
           H8817H10   6.940000        1.0  16116900.0   6.940000 -0.009986
           Y2109Q70   5.650000        1.0    251496.0   5.650000  0.027273
           Y4600220   2.959400        1.0       638.0   2.959400  0.027569

[171105 rows x 5 columns]

In [238]:
stock_dt.to_csv('stock.csv')
factors.to_csv('factors.csv')

# Cross sectional regression with a random selected stock

In [239]:
test = stock_dt.xs('00154710', level='cusip')
reg_model = sm.OLS(test.ret.astype('float'), factors.astype('float')).fit()
display(reg_model.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    ret   R-squared (uncentered):                   0.520
Model:                            OLS   Adj. R-squared (uncentered):              0.491
Method:                 Least Squares   F-statistic:                              18.04
Date:                Sun, 11 Oct 2020   Prob (F-statistic):                   1.54e-114
Time:                        22:36:12   Log-Likelihood:                          2157.4
No. Observations:                1005   AIC:                                     -4201.
Df Residuals:                     948   BIC:                                     -3921.
Df Model:                          57                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0111      0.007      1.516      0.130      -0.003       0.025
SMB            0.0007      0.006      0.122      0.903      -0.011       0.012
HML            0.0023      0.005      0.467      0.640      -0.008       0.012
RMW           -0.0036      0.005     -0.761      0.447      -0.013       0.006
CMA            0.0017      0.005      0.354      0.723      -0.008       0.011
MOM           -0.5848      0.184     -3.173      0.002      -0.946      -0.223
ST_Rev         0.0041      0.002      2.271      0.023       0.001       0.008
LT_Rev        -0.0020      0.004     -0.523      0.601      -0.009       0.005
Agric         -0.0002      0.001     -0.209      0.835      -0.002       0.002
Food          -0.0003      0.002     -0.198      0.843      -0.004       0.003
Soda           0.0005      0.001      0.661      0.509      -0.001       0.002
Beer           0.0001      0.001      0.109      0.913      -0.002       0.002
Smoke       6.129e-05      0.001      0.062      0.951      -0.002       0.002
Toys        3.104e-05      0.001      0.030      0.976      -0.002       0.002
Fun           -0.0004      0.002     -0.210      0.833      -0.004       0.003
Books          0.0003      0.001      0.228      0.820      -0.003       0.003
Hshld         -0.0012      0.002     -0.612      0.540      -0.005       0.003
Clths          0.0014      0.001      1.417      0.157      -0.001       0.003
Hlth          -0.0015      0.001     -1.159      0.247      -0.004       0.001
MedEq         -0.0003      0.002     -0.149      0.881      -0.004       0.004
Drugs          0.0002      0.002      0.125      0.901      -0.003       0.003
Chems          0.0004      0.002      0.203      0.839      -0.003       0.004
Rubbr         -0.0035      0.002     -2.132      0.033      -0.007      -0.000
Txtls         -0.0007      0.001     -0.557      0.577      -0.003       0.002
BldMt         -0.0019      0.002     -0.884      0.377      -0.006       0.002
Cnstr         -0.0011      0.001     -0.765      0.444      -0.004       0.002
Steel          0.0185      0.001     16.157      0.000       0.016       0.021
FabPr          0.0011      0.001      1.667      0.096      -0.000       0.002
Mach          -0.0114      0.003     -4.116      0.000      -0.017      -0.006
ElcEq          0.0007      0.001      0.488      0.626      -0.002       0.003
Autos          0.0010      0.002      0.570      0.569      -0.002       0.004
Aero           0.0002      0.002      0.121      0.904      -0.003       0.003
Ships          0.0019      0.001      1.789      0.074      -0.000       0.004
Guns          -0.0001      0.001     -0.103      0.918      -0.002       0.002
Gold           0.0006      0.000      1.328      0.184      -